In [ ]:
import pandas as pd


/Users/benedikt/Documents/code/data-science-g5/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
path = "../../KW6/polizeimeldungen/data_as_json"

In [3]:
import os

# List all files in the directory
files = [f for f in os.listdir(path) if f.endswith('.json')]

# Read all JSON files into a dataframe
dataframes = [pd.read_json(os.path.join(path, file)) for file in files]

# Concatenate all dataframes into a single dataframe
df = pd.concat(dataframes, ignore_index=True)
display(df)

,date,title,location,link,details,number
0,2021-04-17 13:00:00,Brandsätze gegen Fassade geworfen - Gesuchter ...,Tempelhof-Schöneberg,/polizei/polizeimeldungen/pressemitteilung.100...,Im Zusammenhang mit dem im Oktober des vergang...,0841
1,2021-04-13 10:01:00,Jugendliche nach nächtlichem Überfall im Poliz...,Steglitz-Zehlendorf,/polizei/polizeimeldungen/2021/pressemitteilun...,Nach einem in der vergangenen Nacht gemeinscha...,0801
2,2021-02-09 15:01:00,Betrüger und Trickdiebe als falsche Versicheru...,berlinweit,/polizei/polizeimeldungen/pressemitteilung.102...,Zwischenzeitlich konnte das Landeskriminalamt ...,0316
3,2021-02-04 11:00:00,Gefährliche Körperverletzung,Mitte,/polizei/polizeimeldungen/pressemitteilung.960...,Die Kriminalpolizei der Direktion 2 bittet um ...,0276
4,2021-01-12 13:02:00,Verkehrsunfall mit schwerverletztem E-Bike-Fahrer,Mitte,/polizei/polizeimeldungen/pressemitteilung.103...,Gestern Nachmittag wurde in Mitte bei einem Ve...,0092
...,...,...,...,...,...,...
12219,2023-01-01 11:05:00,Frau bei Überfall auf Spätkauf verletzt,Tempelhof-Schöneberg,/polizei/polizeimeldungen/2023/pressemitteilun...,Ein bislang Unbekannter überfiel in der vergan...,5.0
12220,2023-01-01 11:02:00,Geschlagen und homophob beleidigt,Tempelhof-Schöneberg,/polizei/polizeimeldungen/2023/pressemitteilun...,Unbekannte beleidigten in der vergangenen Nach...,4.0
12221,2023-01-01 10:04:00,Getreten und mit Messer angegriffen,Treptow-Köpenick,/polizei/polizeimeldungen/2023/pressemitteilun...,Im Ortsteil Altglienicke wurde in der vergange...,3.0
12222,2023-01-01 10:04:00,Festnahme nach homophober Beleidigung,Pankow,/polizei/polizeimeldungen/2023/pressemitteilun...,Polizeieinsatzkräfte nahmen gestern Nachmittag...,2.0


# Verbrechen haben eine Saisonalität


In [4]:
from transformers import pipeline
classifier = pipeline("zero-shot-classification",
                      model="Sahajtomar/German_Zeroshot")

Device set to use mps:0


In [5]:
df.set_index('date', inplace=True)

## Wann finden die Verbrechen statt?

In [6]:
import plotly.express as px
from plotly.subplots import make_subplots

# Aggregated data
hourly_counts = df.groupby(df.index.hour).size()
weekly_counts = df.groupby(df.index.isocalendar().week).size()
monthly_counts = df.groupby(df.index.month).size()
yearly_counts = df.resample('Y').size()

# Create subplots
fig = make_subplots(rows=2, cols=2, subplot_titles=('Hourly Counts', 'Weekly Counts', 'Monthly Counts', 'Yearly Counts'))

# Hourly histogram
fig.add_trace(px.bar(hourly_counts, title='Hourly Counts').data[0], row=1, col=1)

# Weekly histogram
fig.add_trace(px.bar(weekly_counts, title='Weekly Counts').data[0], row=1, col=2)

# Monthly histogram
fig.add_trace(px.bar(monthly_counts, title='Monthly Counts').data[0], row=2, col=1)

# Yearly histogram
fig.add_trace(px.bar(yearly_counts, title='Yearly Counts').data[0], row=2, col=2)

fig.update_layout(height=800, width=1200, title_text='Crime Counts Over Different Time Periods')
fig.show()

In [7]:
import spacy

# Schritt 2: Lade das Sprachmodell und definiere die Tageszeiten
nlp = spacy.load("de_core_news_lg")


# Schritt 3: Schreibe eine Funktion, die die Tageszeit aus einem Text extrahiert
def extract_time_of_day(text, keywords_dict):
    doc = nlp(text.lower())
    for token in doc:
        for time_of_day, keywords in keywords_dict.items():
            if token.text in keywords:
                return time_of_day
    return "Unbekannt"


In [11]:
time_of_day_keywords = {
    "Vormittag": ["morgen", "vormittag"],
    "Mittag": ["mittag", "mittags"],
    "Nachmittag": ["nachmittag", "nachmittags"],
    "Abend": ["abend", "abends"],
    "Nacht": ["nacht", "nachts"]
}

df["Tagesabschnitt"] = df["details"].apply(lambda x: extract_time_of_day(x, time_of_day_keywords) if pd.notnull(x) else "Unbekannt")

In [12]:
px.histogram(df, x="Tagesabschnitt", title="Verbrechen nach Tageszeit")

In [35]:
df[df["Tagesabschnitt"]=="Unbekannt"]["details"][0:3].values

array(['Im Zusammenhang mit dem im Oktober des vergangenen Jahres erfolgten Brandanschlag konnte gestern ein 43-Jähriger in Gesundbrunnen nach Hinweisen aus der Bevölkerung festgenommen werden. Nach Feststellung der zuständigen Staatsanwaltschaft lässt sich jedoch der dringende Tatverdacht im Zusammenhang mit dem Brandgeschehen nicht aufrechterhalten. Es wurde folglich die Freilassungsanordnung getroffen und der Festgenommene wieder auf freien Fuß gesetzt. Die Ermittlungen des Polizeilichen Staatsschutzes des Landeskriminalamtes dauern an.',
       'Zwischenzeitlich konnte das Landeskriminalamt im Rahmen intensiver Ermittlungen einen Tatverdächtigen namhaft machen. Der Mann, der bereits früher wegen Betrugstaten polizeilich in Erscheinung getreten war, wurde erst im Oktober des vergangenen Jahres aus der Haft entlassen. Obwohl er seitdem über keine Meldeanschrift verfügte, gelang es den Fahndern, ihn am 4. Februar 2021 festzunehmen. Wegen zwei neu erlassener Haftbefehle wegen Betruges 

In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
import spacy

# Load the German language model
nlp = spacy.load("de_core_news_lg")

# Function to preprocess text (tokenize and lemmatize)
def preprocess_text(text):
    doc = nlp(text)
    return ' '.join([token.lemma_ for token in doc if not token.is_stop and not token.is_punct])

# Apply preprocessing to the 'details' column
df['processed_details'] = df['details'].apply(lambda x: preprocess_text(x) if pd.notnull(x) else "")

# Vectorize the processed text
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(df['processed_details'])

# Perform KMeans clustering
kmeans = KMeans(n_clusters=20, random_state=42)
df['cluster'] = kmeans.fit_predict(tfidf_matrix)

# Display the first few rows of the dataframe with the cluster labels
display(df.head())

,title,location,link,details,number,Tagesabschnitt,processed_details,cluster
date,,,,,,,,
2021-04-17 13:00:00,Brandsätze gegen Fassade geworfen - Gesuchter ...,Tempelhof-Schöneberg,/polizei/polizeimeldungen/pressemitteilung.100...,Im Zusammenhang mit dem im Oktober des vergang...,0841,Unbekannt,Zusammenhang Oktober Jahr erfolgt Brandanschla...,13
2021-04-13 10:01:00,Jugendliche nach nächtlichem Überfall im Poliz...,Steglitz-Zehlendorf,/polizei/polizeimeldungen/2021/pressemitteilun...,Nach einem in der vergangenen Nacht gemeinscha...,0801,Nacht,Nacht gemeinschaftlich begangen Raub jugendlic...,18
2021-02-09 15:01:00,Betrüger und Trickdiebe als falsche Versicheru...,berlinweit,/polizei/polizeimeldungen/pressemitteilung.102...,Zwischenzeitlich konnte das Landeskriminalamt ...,0316,Unbekannt,zwischenzeitlich Landeskriminalamt Rahmen inte...,2
2021-02-04 11:00:00,Gefährliche Körperverletzung,Mitte,/polizei/polizeimeldungen/pressemitteilung.960...,Die Kriminalpolizei der Direktion 2 bittet um ...,0276,Unbekannt,Kriminalpolizei Direktion 2 bitten Mithilfe un...,10
2021-01-12 13:02:00,Verkehrsunfall mit schwerverletztem E-Bike-Fahrer,Mitte,/polizei/polizeimeldungen/pressemitteilung.103...,Gestern Nachmittag wurde in Mitte bei einem Ve...,0092,Nachmittag,Gestern Nachmittag Mitte Verkehrsunfall E-Bike...,0


In [41]:
tfidf_matrix.shape

(12224, 24710)

In [40]:
vectorizer.get_feature_names_out().shape

(24710,)

In [16]:
import numpy as np

# Get the feature names (words) from the vectorizer
terms = vectorizer.get_feature_names_out()

# Get the cluster centers
order_centroids = kmeans.cluster_centers_.argsort()[:, ::-1]

# Print the top 10 words for each cluster
for i in range(kmeans.n_clusters):
    print(f"Cluster {i}:")
    for ind in order_centroids[i, :10]:
        print(f" {terms[ind]}")
    print()

Cluster 0:
 radfahrer
 richtung
 verkehrsunfall
 schwer
 stationär
 stürzen
 krankenhaus
 straße
 jährig
 erleiden

Cluster 1:
 geldautomate
 geldausgabeautomat
 sprengen
 unbekannte
 bankfiliale
 automat
 fachkommissariat
 landeskriminalamt
 früh
 ermittlung

Cluster 2:
 strong
 erstmeldung
 nr
 br
 mithilfe
 bitten
 vermissen
 2023
 2024
 2022

Cluster 3:
 wohnung
 mann
 jährig
 mehrfamilienhaus
 frau
 einsatzkraft
 polizei
 alarmieren
 finden
 ermittlung

Cluster 4:
 fahrer
 geschwindigkeit
 fahren
 fahrzeug
 wagen
 straße
 überhöht
 auto
 polizist
 ampel

Cluster 5:
 fest
 nehmen
 mann
 mutmaßlich
 einsatzkraft
 festnehmen
 tatverdächtig
 polizist
 beobachten
 jährig

Cluster 6:
 gruppe
 person
 mann
 heraus
 auseinandersetzung
 jähriger
 flüchten
 personengruppe
 schlagen
 jährige

Cluster 7:
 motorradfahrer
 richtung
 verkehrsunfall
 motorrad
 verletzung
 jährig
 krankenhaus
 stationär
 erleiden
 straße

Cluster 8:
 flamme
 feuerwehr
 löschen
 brandkommissariat
 brandstiftung
 br

In [18]:

# Create a dictionary to map cluster numbers to their titles
cluster_titles = {
    0: "Schwere Fahrradunfälle und Krankenhausaufenthalte",
    1: "Geldautomatensprengungen und polizeiliche Ermittlungen",
    2: "Vermisstenfälle und Erstmeldungen",
    3: "Polizeieinsätze in Wohnungen und Mehrfamilienhäusern",
    4: "Verkehrsdelikte und Geschwindigkeitsüberschreitungen",
    5: "Festnahmen von Tatverdächtigen durch die Polizei",
    6: "Auseinandersetzungen und Gruppen-Konflikte",
    7: "Motorradunfälle mit schweren Verletzungen",
    8: "Brände und mutmaßliche Brandstiftungen",
    9: "Ermittlungen zu Branddelikten und nächtlichen Feuern",
    10: "Messerangriffe und Gewaltdelikte mit Verletzungen",
    11: "Raubüberfälle auf Geschäfte und Tankstellen",
    12: "Verkehrsunfälle mit Kindern und Jugendlichen",
    13: "Polizeieinsätze bei Versammlungen und Demonstrationen",
    14: "Feuerwehreinsätze bei Bränden und Brandstiftungen",
    15: "Verkehrsunfälle mit Fußgängern und schweren Verletzungen",
    16: "Fremdenfeindliche und rassistische Beleidigungen",
    17: "Sachbeschädigungen mit politischem Hintergrund",
    18: "Jugendkriminalität und Vorfälle mit Jugendlichen",
    19: "Verkehrsunfälle mit Autos und Verletzten"
}

# Map the cluster numbers to their titles
df['cluster_title'] = df['cluster'].map(cluster_titles)


In [42]:
# Map the cluster numbers to their titles
df[df['cluster_title'] == "Geldautomatensprengungen und polizeiliche Ermittlungen"]["details"].head(10).values

array(['Bisher unbekannt gebliebene Täter brachen in der vergangenen Nacht mehrere Fahrzeuge in Charlottenburg-Nord auf. Nach derzeitigem Kenntnisstand bemerkte ein Zeuge gegen 22 Uhr auf einem Parkplatz am Heckerdamm, wie drei Unbekannte mehrere Scheiben von geparkten Autos zerstörten und die Innenräume durchwühlten. Er alarmierte daraufhin die Polizei. Bevor die Einsatzkräfte eintrafen, flüchteten die Unbekannten in Richtung Volkspark Jungfernheide und entkamen. Eine Absuche nach den Tätern verlief ohne Erfolg. Weitere Ermittlungen ergaben, dass insgesamt elf Autos aufgebrochen wurden und die Unbekannten in einem Fall auch Beute, offenbar ein Autoradio, machten. Die weiteren, noch andauernden Ermittlungen hat die Kriminalpolizei der Direktion 2 übernommen.',
       'Unbekannte versuchten vergangene Nacht in Neu-Hohenschönhausen einen Geldautomaten zu öffnen und beschädigten ihn dabei. Nach bisherigen Erkenntnissen hörten Anwohner der Rostocker Straße gegen 2.50 Uhr einen lauten Knall

In [ ]:
px.histogram(df, x='cluster_title', title='Verbrechen nach Uhrzeit')

In [19]:
df[df['cluster'] == 18]["cluster_title"][0]

/var/folders/t5/m580657s5rxblthmnv5y6vfm0000gn/T/ipykernel_65254/3508897768.py:1: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`



'Jugendkriminalität und Vorfälle mit Jugendlichen'

In [20]:
# cluster_title
df[df['cluster']==18]["details"].sample(5).values

array(['Heute früh nahmen Polizeikräfte in Hellersdorf zwei Tatverdächtige nach mehreren Diebstählen aus Autos fest. Nach dem derzeitigen Stand der Ermittlungen bemerkte ein Anwohner gegen 3.45 Uhr zwei Jugendliche auf einem Parkplatz in der Hellersdorfer Straße, die die Scheiben von vier Fahrzeugen eingeschlagen haben sollen. Daraufhin alarmierte er die Polizei. Einsatzkräfte nahmen kurz darauf die beiden 16-Jährigen fest, die gerade einen VW durchwühlten. Im Anschluss brachten Polizeikräfte die Jugendlichen zu erkennungsdienstlichen Behandlungen sowie Blutentnahmen in einen Polizeigewahrsam und übergaben die beiden dann einem Fachkommissariat der Polizeidirektion 3 (Ost). Die weiteren Ermittlungen dauern an.',
       'Gestern früh wurde in Alt-Hohenschönhausen ein Jugendlicher laut eigenen Angaben bei einem Raub verletzt. Nach bisherigen Ermittlungsstand wurde der 17-Jährige gegen 5.25 Uhr in Höhe der Kreuzung Gembitzer Straße Ecke Malchower Weg von drei unbekannt gebliebenen maskier

In [45]:
# title = "Verkehrsdelikte und Geschwindigkeitsüberschreitungen"
title = "Motorradunfälle mit schweren Verletzungen"
# title = "Schwere Fahrradunfälle und Krankenhausaufenthalte"
# Aggregated data
hourly_counts = df[df['cluster_title'] == title].groupby(
    df[df['cluster_title'] == title].index.hour).size()
weekly_counts = df[df['cluster_title'] == title].groupby(
    df[df['cluster_title'] == title].index.isocalendar().week).size()
monthly_counts = df[df['cluster_title'] ==
                    title].groupby(df[df['cluster_title'] == title].index.month).size()
yearly_counts = df[df['cluster_title'] == title].resample('Y').size()

# Create subplots
fig = make_subplots(rows=2, cols=2, subplot_titles=(
    'Hourly Counts', 'Weekly Counts', 'Monthly Counts', 'Yearly Counts'))

# Hourly histogram
fig.add_trace(
    px.bar(hourly_counts, title=f'Hourly {title}').data[0], row=1, col=1)

# Weekly histogram
fig.add_trace(
    px.bar(weekly_counts, title=f'Weekly {title}').data[0], row=1, col=2)

# Monthly histogram
fig.add_trace(
    px.bar(monthly_counts, title=f'Monthly {title}').data[0], row=2, col=1)

# Yearly histogram
fig.add_trace(
    px.bar(yearly_counts, title=f'Yearly {title}').data[0], row=2, col=2)

fig.update_layout(height=800, width=1200,
                  title_text=title + ' Counts Over Different Time Periods')
fig.show()